# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders_df = pd.read_csv("Orders.csv")
orders_df.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [9]:
customer_spent = orders_df.groupby("CustomerID").agg({"amount_spent":"sum"}).reset_index() #group by the unique customers ID by the sum of their amount spent
customer_spent

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


In [23]:
def quantile_range (Q_min, Q_max): #create a function to print list of customers in a given quantile range. I ordered by amount spent.
    spent_Q_min = customer_spent["amount_spent"].quantile(Q_min)
    spent_Q_max = customer_spent["amount_spent"].quantile(Q_max)
    customers_in_range = customer_spent[(customer_spent["amount_spent"] >= spent_Q_min) & (customer_spent["amount_spent"] <= spent_Q_max)]
    return customers_in_range.sort_values(by="amount_spent", ascending=False)


In [30]:
preferred_range = quantile_range(0.75, 0.95) #it uses the function to add all customers between 75% and 95%
customers_vip = quantile_range(0.95, 1) #it uses the function to add all customers vip
customers_preferred = preferred_range[~preferred_range["CustomerID"].isin(customers_vip["CustomerID"])] #it uses the function to determine preferred customers (excluding the +95% ones that are vip)



In [31]:
customers_vip #Vip customers list

,CustomerID,amount_spent
1690,14646,280206.02
4202,18102,259657.30
3729,17450,194550.79
3009,16446,168472.50
1880,14911,143825.06
...,...,...
1587,14505,5905.64
1516,14415,5900.28
1898,14944,5900.25
650,13209,5891.61


In [32]:
customers_preferred #Preferred customers list

,CustomerID,amount_spent
537,13050,5836.86
304,12720,5781.73
2105,15218,5756.89
3904,17686,5739.46
631,13178,5725.47
...,...,...
3880,17656,1674.69
2764,16115,1667.97
438,12912,1662.30
147,12530,1662.28


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [36]:
country_vip = orders_df[orders_df["CustomerID"].isin(customers_vip["CustomerID"])].groupby("Country").agg({"CustomerID":"count"}).reset_index()
country_vip.sort_values(by="CustomerID", ascending=False) #Uk has the most VIP Customers

,Country,CustomerID
17,United Kingdom,84185
5,EIRE,7077
7,France,3290
8,Germany,3127
10,Netherlands,2080
0,Australia,898
12,Portugal,681
16,Switzerland,594
14,Spain,511
11,Norway,420


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [35]:
VipPreferred_countries = orders_df[orders_df["CustomerID"].isin(customers_vip["CustomerID"]) | orders_df["CustomerID"].isin(customers_preferred["CustomerID"])].groupby("Country").agg({"CustomerID":"count"}).reset_index()
VipPreferred_countries #Australia has the most preferred + vip clients

,Country,CustomerID
0,Australia,1028
1,Austria,158
2,Belgium,1557
3,Canada,135
4,Channel Islands,589
5,Cyprus,451
6,Denmark,217
7,EIRE,7238
8,Finland,504
9,France,6301
